In [2]:
#pretrained Embeddings 
from gensim.models import KeyedVectors
filename = "drive/MyDrive/GoogleNews-vectors-negative300.bin.gz"
model = KeyedVectors.load_word2vec_format(filename, binary = True)

In [4]:
 import re
import pandas as pd
import numpy as np
import nltk
nltk.download('all')
stop_words = nltk.corpus.stopwords.words('english')

def normalize_corpus(doc):
  doc = re.sub(r'[^a-zA-Z0-9/s]',' ',doc, re.I | re.A)
  doc = doc.lower()
  doc = doc.strip()
  tokens = nltk.word_tokenize(doc)
  filtered_toks = [token for token in tokens if token not in stop_words]
  doc = " ".join(filtered_toks)
  return doc

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping corpora/biocreative_ppi.zip.
[nltk_data]    | Downloadin

In [3]:
model

In [4]:
result = model.most_similar(positive = ['woman','king'], negative = ['man'],topn=1)
print(result)

[('queen', 0.7118192911148071)]


In [ ]:
model.wv['car']

In [ ]:
model.wv.most_similar('car')

In [ ]:
model.wv.most_similar('car',topn = 50)

In [22]:
import tensorflow as tf
from keras.models import Sequential 
from keras.preprocessing.text import Tokenizer,one_hot
from keras.layers import Dense, Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing.sequence import pad_sequences

In [2]:
doc = ['Good Job!',"Excellent Efforts",'Well Done!','nice work','Perfect!',
       'Bad!','Poor','Weak!','Very disappointed','Unsatisfactory']

In [5]:
labels = np.array([1,1,1,1,1,0,0,0,0,0])

In [7]:
t = Tokenizer()
t.fit_on_texts(doc)
vocab_size = len(t.word_index)+1
encoded = t.texts_to_sequences(doc)

In [8]:
encoded

[[1, 2], [3, 4], [5, 6], [7, 8], [9], [10], [11], [12], [13, 14], [15]]

In [10]:
max_length = 2
padded_documents = pad_sequences(encoded,maxlen = max_length, padding = 'post')

In [11]:
padded_documents

array([[ 1,  2],
       [ 3,  4],
       [ 5,  6],
       [ 7,  8],
       [ 9,  0],
       [10,  0],
       [11,  0],
       [12,  0],
       [13, 14],
       [15,  0]], dtype=int32)

In [13]:
f = open("drive/MyDrive/Glove/glove.6B.100d.txt")
embeddings_index = dict()
for line in f: 
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype = 'float32')
  embeddings_index[word] = coefs
f.close()

In [16]:
embeddings_index['good']

array([-0.030769 ,  0.11993  ,  0.53909  , -0.43696  , -0.73937  ,
       -0.15345  ,  0.081126 , -0.38559  , -0.68797  , -0.41632  ,
       -0.13183  , -0.24922  ,  0.441    ,  0.085919 ,  0.20871  ,
       -0.063582 ,  0.062228 , -0.051234 , -0.13398  ,  1.1418   ,
        0.036526 ,  0.49029  , -0.24567  , -0.412    ,  0.12349  ,
        0.41336  , -0.48397  , -0.54243  , -0.27787  , -0.26015  ,
       -0.38485  ,  0.78656  ,  0.1023   , -0.20712  ,  0.40751  ,
        0.32026  , -0.51052  ,  0.48362  , -0.0099498, -0.38685  ,
        0.034975 , -0.167    ,  0.4237   , -0.54164  , -0.30323  ,
       -0.36983  ,  0.082836 , -0.52538  , -0.064531 , -1.398    ,
       -0.14873  , -0.35327  , -0.1118   ,  1.0912   ,  0.095864 ,
       -2.8129   ,  0.45238  ,  0.46213  ,  1.6012   , -0.20837  ,
       -0.27377  ,  0.71197  , -1.0754   , -0.046974 ,  0.67479  ,
       -0.065839 ,  0.75824  ,  0.39405  ,  0.15507  , -0.64719  ,
        0.32796  , -0.031748 ,  0.52899  , -0.43886  ,  0.6740

In [14]:
embedding_matrix = np.zeros((vocab_size,100))

In [15]:
for word, i in t.word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None: 
    embedding_matrix[i]= embedding_vector

In [17]:
model = Sequential()
e = Embedding(vocab_size,100,weights = [embedding_matrix], input_length=2,trainable = False)
model.add(e)
model.add(Flatten())
model.add(Dense(1,activation = 'sigmoid'))

In [18]:
model.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy'])

In [19]:
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 2, 100)            1600      
                                                                 
 flatten (Flatten)           (None, 200)               0         
                                                                 
 dense (Dense)               (None, 1)                 201       
                                                                 
Total params: 1,801
Trainable params: 201
Non-trainable params: 1,600
_________________________________________________________________


In [20]:
model.fit(padded_documents,labels,epochs = 50,verbose = 1)

Epoch 1/50
1/1 [==============================] - 1s 859ms/step - loss: 0.6350 - accuracy: 0.7000
Epoch 2/50
1/1 [==============================] - 0s 10ms/step - loss: 0.6242 - accuracy: 0.7000
Epoch 3/50
1/1 [==============================] - 0s 8ms/step - loss: 0.6136 - accuracy: 0.8000
Epoch 4/50
1/1 [==============================] - 0s 6ms/step - loss: 0.6031 - accuracy: 0.8000
Epoch 5/50
1/1 [==============================] - 0s 7ms/step - loss: 0.5928 - accuracy: 0.8000
Epoch 6/50
1/1 [==============================] - 0s 8ms/step - loss: 0.5827 - accuracy: 0.8000
Epoch 7/50
1/1 [==============================] - 0s 7ms/step - loss: 0.5728 - accuracy: 0.8000
Epoch 8/50
1/1 [==============================] - 0s 9ms/step - loss: 0.5630 - accuracy: 0.8000
Epoch 9/50
1/1 [==============================] - 0s 12ms/step - loss: 0.5534 - accuracy: 0.8000
Epoch 10/50
1/1 [==============================] - 0s 8ms/step - loss: 0.5439 - accuracy: 0.8000
Epoch 11/50
1/1 [==================

In [23]:
model.predict(pad_sequences([one_hot('great work', vocab_size)], maxlen=max_length, padding='post'))

array([[0.8296336]], dtype=float32)